In [3]:
%reload_ext autoreload
%autoreload 2

from lkf_tools.dataset import *
import dill
import matplotlib.pyplot as plt
from datetime import datetime

In [4]:
path = "data/lkf_datasets/"
lkfs_f_1950 = dill.load(open(path + "ice_fesom_1950.pickle", "rb"))
lkfs_f_1951 = dill.load(open(path + "ice_fesom_1951.pickle", "rb"))
lkfs_f_1952 = dill.load(open(path + "ice_fesom_1952.pickle", "rb"))
lkfs_f_1953 = dill.load(open(path + "ice_fesom_1953.pickle", "rb"))
lkfs_f_1954 = dill.load(open(path + "ice_fesom_1954.pickle", "rb"))
lkfs_f_1955 = dill.load(open(path + "ice_fesom_1955.pickle", "rb"))
lkfs_f_1956 = dill.load(open(path + "ice_fesom_1956.pickle", "rb"))

In [5]:
lkf_data = lkfs_f_1956
year = 1956

In [4]:
for i in range(len(lkf_data.indexes)):
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs

    fig = plt.figure(figsize=[16, 8])

    ax = plt.subplot(1, 1, 1, projection=ccrs.Orthographic(0, 90))

    ax.coastlines(zorder=3)

    it = lkf_data.indexes[i]

    pcm = ax.pcolormesh(lkf_data.lon[max([0,lkf_data.index_y[0][0]-1]):lkf_data.index_y[0][-1]+2:lkf_data.red_fac,
                           max([0,lkf_data.index_x[0][0]-1]):lkf_data.index_x[0][-1]+2:lkf_data.red_fac],
                  lkf_data.lat[max([0,lkf_data.index_y[0][0]-1]):lkf_data.index_y[0][-1]+2:lkf_data.red_fac,
                           max([0,lkf_data.index_x[0][0]-1]):lkf_data.index_x[0][-1]+2:lkf_data.red_fac],
                  lkf_data.calc_eps_tot(it),transform=ccrs.PlateCarree(),vmin=0,vmax=0.4)

    lkfs = np.load(lkf_data.lkfpath.joinpath('lkf_%s_%03i.npy' %(lkf_data.netcdf_file.split('/')[-1].split('.')[0],(it+1))),allow_pickle=True)

    for ilkf in lkfs:
        if np.min(ilkf[:,2])<-150 and np.max(ilkf[:,2]>150):
            ilkf[ilkf[:,2]<0,2]+=360
        ax.plot(ilkf[:,2],ilkf[:,3],transform=ccrs.PlateCarree())
        
    day = str(lkf_data.indexes[i]+1)
    day.rjust(3 + len(day), '0')
    year = f"{year}"
    res = datetime.strptime(year + "-" + day, "%Y-%j").strftime("%m-%d-%Y")
        
    ax.text(0.03, 0.97, f'{res}', transform=ax.transAxes, fontsize=14,
        verticalalignment='top')

    #plt.rcParams.update({'font.size': 14})
    #plt.colorbar(pcm,label='deformation rate/ day$^-1$', location='bottom', shrink=0.5, pad=0.02)
    plt.savefig(f'imgs/fesom/{year}/{i}.png',dpi=200,bbox_inches='tight')
    plt.close(fig)

In [5]:
import glob
from PIL import Image
def make_gif(frame_folder):
    frames = []
    for i in range(len(lkf_data.indexes)):
        frames.append(Image.open(frame_folder + f'/{i}.png'))
    frame_one = frames[0]

    frame_one.save(f"gifs/f_{year}.gif", format="GIF", append_images=frames,
                       save_all=True, duration=250, loop=1)

if __name__ == "__main__":
    make_gif(f"imgs/fesom/{year}")